In [433]:
import pandas as pd
import numpy as np
import importlib
import plotly.express as px
import pandas as pd
import utilities.utility as utility
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from pypfopt import EfficientFrontier, risk_models, expected_returns, plotting
importlib.reload(utility)

<module 'utilities.utility' from '/Users/herbishtini/Documents/UNI/Master Thesis/sustainability_portfolio_optimisation/utilities/utility.py'>

In [434]:
df = pd.read_csv('../data/10_monthly_returns_complete.csv')
df_close = pd.read_csv('../data/10_monthly_adjacent_close.csv', index_col=0)
df_overview = pd.read_csv('../data/data_10_overview.csv', index_col=0)

In [435]:
df_overview['return_rate_1y_avg'] = df_overview['return_rate_1y_avg'] - 1
df_overview['return_rate_5y_avg'] = df_overview['return_rate_5y_avg'] - 1
df_overview['return_rate_10y_avg'] = df_overview['return_rate_10y_avg'] - 1
df_overview['return_rate_25y_avg'] = df_overview['return_rate_25y_avg'] - 1

In [436]:
df_overview = df_overview.dropna()

## 1. Equities: risk-return

In [454]:
df_risk_return = df_overview.sort_values(by='market_capital_euro', ascending=False)
df_risk_return = df_risk_return[['stock_ticker_symbol', 'return_rate_25y_avg', 'return_rate_5y_avg', 'return_rate_10y_avg', 'return_rate_1y_avg', 'volatility_5y']]
# Create a scatter plot
fig = px.scatter(df_risk_return, 
                 x='volatility_5y', 
                 y='return_rate_5y_avg', 
                 text='stock_ticker_symbol',
                 title='Risk-Return Plot',
                 labels={'volatility_5y': 'Risk (Standard Deviation)',
                         'return_rate_5y_avg': 'Investment Return Average (last 5 years)'},
                 template='plotly')

# Add labels to the points
fig.update_traces(textposition='top center')

# Show the plot
fig.show()

## 2. Equities: efficient-frontier

### 2.1 Using PyPortfolioOpt library (@TODO make functional)

In [443]:
df_2 = pd.read_csv('../data/10_monthly_adjacent_close.csv', parse_dates=True, index_col='Date')
df_2.index.names = ['date']

# Calculate expected annual returns and sample covariance matrix
date = pd.Timestamp.today() - pd.DateOffset(years=5)
df_2 = df_2.loc[ df_2.index >= date]

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df_2)
S = risk_models.sample_cov(df_2)

In [458]:
mu

KE        7.234243
SCS      -0.107773
HNI       9.449578
AVT       5.116670
ACCO     -0.806263
           ...    
HALO    312.768716
MATW     -0.469461
KVHI     -0.966829
NEO      -0.852184
UNP       7.249665
Length: 957, dtype: float64

In [457]:
df_overview

,company_name,industry,stock_exchange,stock_ticker_symbol,market_capital_euro,score,return_rate_1y_avg,return_rate_5y_avg,return_rate_10y_avg,return_rate_25y_avg,volatility_1y,volatility_5y,volatility_10y,volatility_25y
0,"Kimball Electronics, Inc.",Technology Hardware,NAS,KE,5.481967e+08,0.000120,-0.197940,0.135967,0.077999,0.030499,0.075240,0.121466,0.108617,0.108617
1,"Steelcase, Inc.",Commercial Services,NYS,SCS,1.461633e+09,0.013235,0.597900,0.025098,0.028514,0.034115,0.090186,0.118895,0.108954,0.106236
2,HNI Corp.,Commercial Services,NYS,HNI,2.138245e+09,0.015005,0.702666,0.159246,0.072259,0.066182,0.089088,0.087072,0.096107,0.101771
3,"Avnet, Inc.",Technology Hardware,NAS,AVT,4.517528e+09,0.018784,0.060338,0.065953,0.037491,0.048292,0.064486,0.093569,0.081193,0.108015
4,ACCO Brands Corp.,Commercial Services,NYS,ACCO,4.399426e+08,0.019449,0.026601,-0.067298,-0.014668,-0.057194,0.093098,0.135272,0.123144,0.249993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,"Halozyme Therapeutics, Inc.",Pharmaceuticals,NAS,HALO,6.410791e+09,0.249868,0.336381,0.279408,0.197747,0.109757,0.116140,0.129228,0.143864,0.193229
955,Matthews International Corp.,Commercial Services,NAS,MATW,7.831761e+08,0.250174,-0.316584,0.015532,-0.031241,0.040695,0.098510,0.098342,0.089769,0.075885
956,"KVH Industries, Inc. (Delaware)",Technology Hardware,NAS,KVHI,8.416451e+07,0.250012,-0.099385,-0.125418,-0.090931,0.029404,0.073585,0.096023,0.098989,0.143995
957,"NeoGenomics, Inc.",Pharmaceuticals,NAS,NEO,1.821173e+09,0.250415,-0.005347,-0.097493,0.111215,0.156116,0.125032,0.167200,0.144416,0.170306


In [446]:
# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
ef.add_constraint(lambda w: w[0] >= 0.2)
ef.add_constraint(lambda w: w[2] == 0.15)
ef.add_constraint(lambda w: w[3] + w[4] <= 0.10)

### 2.2 Using custom-built efficient-frontier method

In [453]:
def plot_efficient_frontier(df):
    # Create the figure
    fig = go.Figure()

    # Add the Efficient Frontier line
    fig.add_trace(go.Scatter(
        x=portfolio_volatilities,
        y=target_returns,
        mode='lines',
        name='Efficient Frontier',
        line=dict(color='green', width=3)
    ))

    # Add the individual assets as points
    fig.add_trace(go.Scatter(
        x=df['volatility_5y'],
        y=df['return_rate_5y_avg'],
        mode='markers',
        name='Assets',
        fillcolor='purple'
    ))

    # Update layout
    fig.update_layout(
        title='Efficient Frontier',
        xaxis_title='Volatility (Risk)',
        yaxis_title='Expected Return',
        legend=dict(x=0, y=1),
        hovermode='closest',
        template='plotly'
    )

    # Add labels to the points
    fig.update_traces(textposition='top center')
    
    fig.show()


target_returns, portfolio_volatilities = utility.efficient_frontier(df_overview) # circa 12 Minutes
# Plot the Efficient Frontier
plot_efficient_frontier(df_overview)

## 3. Industry historical return rate

In [449]:
# Convert 'date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Melt the returns dataframe to long format
df_long_returns = df.melt(id_vars=['Date'], var_name='stock_ticker_symbol', value_name='return_rate_5y')

# Merge the returns and industries dataframes
df_merged = pd.merge(df_long_returns, df_overview, on='stock_ticker_symbol')

# Resample to monthly and calculate mean return for each ticker
df_merged.set_index('Date', inplace=True)

df_i_share_ratio = df_merged[['industry', 'return_rate_5y', 'volatility_5y']].groupby(['industry']).resample('ME').mean()
df_i_share_ratio

ValueError: Invalid frequency: ME

Creating separate columns from indices

In [21]:
df_i_share_ratio = df_i_share_ratio.reset_index(level=0)
df_i_share_ratio = df_i_share_ratio.reset_index(level=0)

In [22]:
# Create line plot
fig = px.line(df_i_share_ratio, 
              x='Date', 
              y='return_rate_5y',
              color='industry', 
              title='Industry historical returns',
              labels={'Date': 'Date', 'return_rate_5y': 'Return Rate'})

# Show the plot
fig.show()

## 4. Industry correlation

In [23]:
# Pivot the DataFrame so that each industry is a column and the rows represent dates
df_industry_heatmap = df_i_share_ratio[['Date', 'industry', 'return_rate_5y']].pivot(index='Date', columns='industry', values='return_rate_5y')

# Ensure the columns are in a consistent order (optional)
df_industry_heatmap = df_industry_heatmap.sort_index(axis=1)

# Calculate the correlation matrix
correlation_matrix = df_industry_heatmap.corr()

In [352]:
df_industry_heatmap

NameError: name 'df_industry_heatmap' is not defined

In [24]:
# Create the heatmap
fig = px.imshow(correlation_matrix,
                # labels=dict(x="Industry", y="Industry", color="Correlation"),
                x=correlation_matrix.columns,
                y=correlation_matrix.index,
                color_continuous_scale="RdBu",
                title="Industry Correlation Heatmap")

fig.layout.height = 1000
fig.layout.width = 1250

# Show the figure
fig.show()